In [1]:
import pandas as pd
import numpy as np
import ast
import nltk
import re
from unicodedata import normalize

file = pd.read_csv("tweet.csv")
file.drop('Unnamed: 0',axis=1,inplace=True)
file.drop('id',axis=1,inplace=True)

In [2]:
file.head()

,Tweet,j_user,created_at,entities,followers_user,FC,RT,trending_topic
0,Desde el #FCJumilla damos nuestra más sincera ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:58:44,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,18,4,0.0
1,Me estoy volviendo loca hasta por cada una de ...,"{'id': 713359736, 'id_str': '713359736', 'name...",2018-09-24 03:05:47,"{'hashtags': [], 'symbols': [], 'user_mentions...",531,0,0,0.0
2,🏟 | El #FCJumilla ya está en La Condomina ⚽️...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:38:18,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,4,2,0.0
3,📹 | El vestuario del #FCJumilla en La Condomin...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:19:29,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,21,6,0.0
4,📑 Jornada 6 | El colegiado andaluz Guzmán Mans...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:10:30,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",7162,0,0,0.0


In [3]:
print(file['entities'][0])
#ens quedem amb:
#hashtags
#user_mentions

{'hashtags': [{'text': 'FCJumilla', 'indices': [9, 19]}], 'symbols': [], 'user_mentions': [{'screen_name': 'alejanvalverde', 'name': 'alejandro valverde', 'id': 248879448, 'id_str': '248879448', 'indices': [100, 115]}], 'urls': [{'url': 'https://t.co/XqLolr5sfl', 'expanded_url': 'https://twitter.com/i/web/status/1046429145914384386', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}


In [4]:
numl = len(file.index.values)
file['hashtags']=np.zeros(numl)
file['hashtags_text']=np.zeros(numl)
file['user_mentions']=np.zeros(numl)
file['urls']=np.zeros(numl)

for line in range(numl):
    file['entities'][line] =ast.literal_eval(file['entities'][line])
    file['hashtags'][line] = file['entities'][line]['hashtags'] 
    file['user_mentions'][line] = file['entities'][line]['user_mentions']
    file['urls'][line] = file['entities'][line]['urls']
    if int(len(str(file['hashtags'][line]))>10):
        file['hashtags_text'][line] = str(file['hashtags'][line]).split(':')[1].split(',')[:-1]

C:\Users\Nuria\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Nuria\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Nuria\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Nuria\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: Sett

In [5]:
file.drop('entities',axis=1,inplace=True)
file.drop('hashtags',axis=1,inplace=True)
file['hashtags_text'] = str(file['hashtags_text'])

In [1]:
# Creem variable numerica amb l'hora del tweet
file['hour'] = np.zeros(numl)
for i in range(len(file)):
    file['hour'][i] = int(file['created_at'][i][11:13])

NameError: name 'np' is not defined

In [ ]:
print(file.head())
file.to_csv('temporal1_tweet.csv')

In [1]:
import pandas as pd
import numpy as np
import ast
import nltk
import re
from unicodedata import normalize

file = pd.read_csv("temporal1_tweet.csv")
#file['hashtags_text'] = str(file['hashtags_text'])

In [2]:
# Substituim per 0 els registres amb NaNs

file[file["trending_topic"].isnull()]
#file= file.drop(file[file["trending_topic"].isnull()].index,axis= 0) 
file = file.fillna(value={"trending_topic":0}) 

In [3]:
elim = pd.read_excel('eliminar_paraules.xlsx', header=None)
y=elim[0]
for i in range(len(elim)):
    y[i] = re.sub(
            r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
            normalize( "NFD", y[i]), 0, re.I)
y = y.tolist()

In [4]:
# Treballem amb una mostra per agilitzar els calculs
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer = CountVectorizer(min_df=0.0001,                            
 stop_words=y, 
 strip_accents='unicode')
X_1 = vectorizer.fit_transform(file['Tweet'])
X_1 = pd.DataFrame(X_1.todense())
X_2 = vectorizer.fit_transform(file['hashtags_text'])
X_2 = pd.DataFrame(X_2.todense())
X_3 = file[['hour', 'followers_user', 'trending_topic']]

X = pd.concat([X_1, X_2, X_3], axis=1)

y_fc = file['FC']
y_rt = file['RT']

print(y_fc.head())
print(y_rt.head())
print(X.head())

0    18
1     0
2     4
3    21
4     0
Name: FC, dtype: int64
0    4
1    0
2    2
3    6
4    0
Name: RT, dtype: int64
   0  1  2  3  4  5  6  7  8  9       ...        58  59  60  61  62  63  64  \
0  0  0  0  0  0  0  0  0  0  0       ...         1   1   1   1   8   1   1   
1  0  0  0  0  0  0  0  0  0  0       ...         1   1   1   1   8   1   1   
2  0  0  1  0  0  0  0  0  0  0       ...         1   1   1   1   8   1   1   
3  0  0  0  0  0  0  0  0  0  0       ...         1   1   1   1   8   1   1   
4  0  0  1  0  0  0  0  0  0  0       ...         1   1   1   1   8   1   1   

   hour  followers_user  trending_topic  
0  15.0            7162             0.0  
1   3.0             531             0.0  
2  15.0            7162             0.0  
3  15.0            7162             0.0  
4  15.0            7162             0.0  

[5 rows x 3970 columns]


In [5]:
print(X.shape)
print(y_rt.shape)
print(y_fc.shape)

(4289, 3970)
(4289,)
(4289,)


In [7]:
X.to_csv('X_tweet_nuria.csv')
y_fc.to_csv('y_fc_tweet_nuria.csv', index = None)
y_rt.to_csv('y_rt_tweet_nuria.csv', index = None)

In [58]:
from sklearn import model_selection

X_train_fc, X_test_fc, y_train_fc, y_test_fc = model_selection.train_test_split(X, y_fc, train_size = 0.7, random_state = 42)

C:\Users\Nuria\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [61]:
# Let us tokenize the data
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(X_train_fc,y_train_fc)

y_hat_fc = nb.predict(X_test_fc)
from sklearn import metrics   
    
print ("classification accuracy:", metrics.accuracy_score(y_hat_fc, y_test_fc))
#print ("Classification Report:")
#print (metrics.classification_report(y_hat_fc,np.array(y_test_fc)))

classification accuracy: 0.46386946386946387


In [63]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors = 10)
clf.fit(X_train_fc, y_train_fc)
y_hat_fc = clf.predict(X_test_fc)
print ("classification accuracy:", metrics.accuracy_score(y_hat_fc, y_test_fc))

classification accuracy: 0.4794094794094794


In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
clf = RandomForestClassifier(n_estimators = 11)

clf.fit(X_train_fc,y_train_fc)
    
y_hat_fc = clf.predict(X_test_fc)
print ("classification accuracy:", metrics.accuracy_score(y_hat_fc, y_test_fc))

classification accuracy: 0.4716394716394716


In [66]:
from sklearn import neighbors, linear_model, ensemble, svm, model_selection
clf = svm.LinearSVC()
clf.fit(X_train_fc,y_train_fc)
    
y_hat_fc = clf.predict(X_test_fc)
print ("classification accuracy:", metrics.accuracy_score(y_hat_fc, y_test_fc))

classification accuracy: 0.4755244755244755


In [67]:
clf = linear_model.LogisticRegression()
clf.fit(X_train_fc,y_train_fc)
    
y_hat_fc = clf.predict(X_test_fc)
print ("classification accuracy:", metrics.accuracy_score(y_hat_fc, y_test_fc))

classification accuracy: 0.47785547785547783


In [82]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 12)
clf.fit(X_train_fc, y_train_fc)
y_hat_fc = clf.predict(X_test_fc)
print ("classification accuracy:", metrics.accuracy_score(y_hat_fc, y_test_fc))

classification accuracy: 0.48484848484848486


In [62]:

X_train_rt, X_test_rt, y_train_rt, y_test_rt = model_selection.train_test_split(X, y_rt, train_size = 0.7, random_state = 42)
nb = BernoulliNB()
nb.fit(X_train_rt,y_train_rt)

y_hat_rt = nb.predict(X_test_rt)
from sklearn import metrics   
    
print ("classification accuracy:", metrics.accuracy_score(y_hat_rt, y_test_rt))
#print ("Classification Report:")
#print (metrics.classification_report(y_hat_rt,np.array(y_test_rt)))

C:\Users\Nuria\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


classification accuracy: 0.7832167832167832


In [72]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 10)
clf.fit(X_train_rt,y_train_rt)

y_hat_rt = clf.predict(X_test_rt)
print ("classification accuracy:", metrics.accuracy_score(y_hat_rt, y_test_rt))

classification accuracy: 0.7902097902097902


In [73]:
clf = RandomForestClassifier(n_estimators = 11)
clf.fit(X_train_rt,y_train_rt)

y_hat_rt = clf.predict(X_test_rt)
print ("classification accuracy:", metrics.accuracy_score(y_hat_rt, y_test_rt))

classification accuracy: 0.7746697746697747


In [74]:
clf = svm.LinearSVC()
clf.fit(X_train_rt,y_train_rt)

y_hat_rt = clf.predict(X_test_rt)
print ("classification accuracy:", metrics.accuracy_score(y_hat_rt, y_test_rt))

classification accuracy: 0.6993006993006993


In [75]:
clf = linear_model.LogisticRegression()
clf.fit(X_train_rt,y_train_rt)

y_hat_rt = clf.predict(X_test_rt)
print ("classification accuracy:", metrics.accuracy_score(y_hat_rt, y_test_rt))

classification accuracy: 0.7832167832167832


In [81]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 9)
clf.fit(X_train_rt,y_train_rt)

y_hat_rt = clf.predict(X_test_rt)
print ("classification accuracy:", metrics.accuracy_score(y_hat_rt, y_test_rt))

classification accuracy: 0.7855477855477856


In [80]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 12)
clf.fit(X_train_rt,y_train_rt)

y_hat_rt = clf.predict(X_test_rt)
print ("classification accuracy:", metrics.accuracy_score(y_hat_rt, y_test_rt))

classification accuracy: 0.7917637917637917
